In [1]:
import os
import pandas as pd
import faiss
import numpy as np
from sklearn.preprocessing import normalize
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
import cv2

In [2]:
train = pd.read_csv("train_ref.csv")
train_noaug = train[train["source"].isna()]
test = pd.read_csv("test_ref.csv")
class_indices = { v:k for k, v in enumerate(pd.Categorical(train["label_name"]).categories) }
indices_classes = { k:v for k, v in enumerate(pd.Categorical(train["label_name"]).categories) }

In [3]:
train_gnd = [ class_indices[x] for x in train["label_name"] ]
train_noaug_gnd = [ class_indices[x] for x in train_noaug["label_name"] ]
test_gnd =  [ class_indices[x] for x in test["label_name"] ]

In [4]:
train_name = train_noaug["name"]
test_name = test["name"]

## I) Retrieval

In [181]:
feature_pool = np.concatenate((
        np.load("./features/patchs_train_noaug.npy"), 
        np.load("./features/features_train_ref_noaug_icon50.npy"),
        np.load("./features/v5_train_ref.npy"),
        np.load("./features/v4_train_ref.npy")
    ), 1)

feature_query = np.concatenate((
        np.load("./features/patchs_test.npy"), 
        np.load("./features/features_test_ref_icon50.npy"),
        np.load("./features/v5_test_ref.npy"),
        np.load("./features/v4_test_ref.npy"),
    ), 1)


In [184]:
index = faiss.IndexFlatIP(feature_query.shape[1])
index.add(normalize(feature_pool))
D, I = index.search(normalize(feature_query), 100)

preds = []
for k, (i, d) in enumerate(zip(I, D)):
    labels = [ train_noaug_gnd[it] for it in i[:10] ]
    pred = Counter(labels).most_common()[0][0]
    preds.append(pred)
    
            
preds = np.array(preds)

In [185]:
print("F1_score macro:", f1_score(test_gnd, preds, average="macro"))
print("F1_score macro:", f1_score(test_gnd, preds, average="weighted"))
print("Accuracy: ", accuracy_score(test_gnd, preds))

F1_score macro: 0.8111844685917678
F1_score macro: 0.8999383188607427
Accuracy:  0.9022869022869023
